# Design Issues in Noninferiority/Equivalance Trials

**Goal**: Generate sample size estimates or power for non-inferiority equivalance trials for either normally or binomially distributed datasets.

**Background.** This repository provides sample size and power estimates for various trial types: non-inferiority, equivalence, superiority, and superiority to placebo, assuming normally distributed data. Additionally, it offers estimates for non-inferiority and equivalence trials with binomially distributed data. For a comprehensive guide on when and how to apply the sample size and power calculations presented here, please refer to Hwang and Moriwaka (1999) [https://doi.org/10.1177/009286159903300424].

**Usage of the sample size and Power Estimation Function**

To use the function below, the user must specify the trial type, distribution, delta, sigma, alpha, and either sample size (n) or power, but not both.

Definition of Variables:

- **Trial:** Noninferiority, Equivalence, Superiority and superiority-to-placebo 
- **Distribution:** Normal or Binomial
- **delta:** Noninferiority or Equivalence margin 
- **sigma:** Standard deviation 
- **Alpha ($\alpha$):** Type I error 
- **n:** Sample size 
- **power (1-$\pi$):** Noninferiority, Equivalence, Superiority and superiority-to-placebo 
 

**Formulas**

**1. Non-inferiority Trial**

**Sample size**

- n = 2 $[[z_{1-\alpha} + z_{1-\beta}]({\sigma}/{\delta})]^{2}$
 
**Power**

power = 2 $\Phi[\delta (2\sigma^{2}/n) ^{-1/2} - z_{1-\alpha}]$


**2. Equivalence Trial**

**Sample size**

- n = 2 $[[z_{1-\alpha} + z_{1-\beta/2}]({\sigma}/{\delta})]^{2}$
 
**Power**

power = 2 $\Phi[\delta (2\sigma^{2}/n) ^{-1/2} - z_{1-\alpha}]-1$

**3. Superiority Trial**

**Sample size**

- n = 2 $[[z_{1-\alpha} + z_{1-\beta}]({\sigma}/{\epsilon})]^{2}$
 
**Power**

power = $\Phi[\epsilon (2\sigma^{2}/n) ^{-1/2} - z_{1-\alpha}]$

**4. Superiority to Placebo Trial**

**Sample size**

- n = 2 $[[z_{1-\alpha} + z_{1-\beta}]({\sigma}/{\Delta})]^{2}$
 
**Power**

power = $\Phi[\Delta (2\sigma^{2}/n) ^{-1/2} - z_{1-\alpha}]$
 
 

**Sample size and Power Estimation Function**

In [146]:
# libraries
import math
from scipy.stats import norm
import pandas as pd

def func_trial_description(trial_var):
    ''''
    Trial description
    '''
    if trial_var == "noninfer":
        trial_description = "Non-Inferiority"
    elif trial_var == "equiv":
        trial_description = "Equivalence"
    elif trial_var == "super":
        trial_description = "Superiority"
    elif trial_var == "supertoplacebo":
        trial_description = "Superiority to Placebo" 

    return(trial_description)

def func_distribution_description(distr_var):
    ''''
    Distribution description
    '''
    if distr_var == "normal":
        distr_description = "normally distributed"
    elif distr_var == "binomial":
        distr_description = "binomially distributed"
    return(distr_description)
 
def func_res_dsn(trial, distr,  delta, sigma, alpha, pi_val, n,  power):
     '''
     Ouput dataset
     '''
     dsn = pd.DataFrame({'trial': trial, 'delta': delta, 'sigma': sigma, 'alpha':alpha,'pi_val': pi_val,  'power': power, "n":n}, index=[1])
     return(dsn)


def func_calc(trial, distr,  delta, sigma, alpha, pi_val, n,  power):
    ''''
    Given sample size (n) - estimate power
    Given power - estimate sample size (n)
    '''
    if power>0:
        beta = 1 - power

    if (trial == "noninfer" and distr=="normal"):

        factor1 = norm.ppf(1 - alpha) 

        if math.isnan(n):

            factor2 = norm.ppf(1 - beta) 
            n = 2*((factor1 + factor2)*(sigma/delta) )**2
            print("Estimated sample size (n): " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            res = func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power)
            return(res)
      
        elif math.isnan(power):

            factor2 =  delta*(   ((2*sigma**2/n))**(-1/2)  )
            diff = factor2 - factor1
            power = norm.cdf(diff, loc=0, scale=1) * 100
            print("Estimated power: " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))
        
    if (trial == "equiv" and distr=="normal"):

        factor1 = norm.ppf(1 - alpha) 

        if math.isnan(n):

            factor2 = norm.ppf(1 - beta/2) 
            n = 2*((factor1 + factor2)*(sigma/delta) )**2
            print("Estimated sample size (n): " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))
      
        elif math.isnan(power):

            factor2 = delta*(   ((2*sigma**2/n))**(-1/2)  )
            diff = factor2 - factor1
            power =( 2 * (norm.cdf(diff, loc=0, scale=1) ) - 1 ) * 100
            print("Estimated power: " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))       

    if (trial == "super" and distr=="normal"):

        factor1 = norm.ppf(1 - alpha) 
    
        if math.isnan(n):

            factor2 = norm.ppf(1 - beta) 
            n = 2*((factor1 + factor2)*(sigma/delta) )**2
            print("Estimated sample size (n): " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))
      
        elif math.isnan(power):

            factor2 =delta * ((2*sigma**2)/n)**(-1/2)
            diff = factor2 - factor1
            power = norm.cdf(diff, loc=0, scale=1) * 100
            print("Estimated power: " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))

    if (trial == "supertoplacebo" and distr=="normal"):

        factor1 = norm.ppf(1 - alpha) 
        print("here")
        if math.isnan(n):

            factor2 = norm.ppf(1 - beta) 
            n = 2*((factor1 + factor2)*(sigma/delta) )**2
            print("Estimated sample size (n): " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))
    
        elif math.isnan(power):


            factor2 =delta * ((2*sigma**2)/n)**(-1/2)
            diff = factor2 - factor1
            power = norm.cdf(diff, loc=0, scale=1) * 100
            print("Estimated power: " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))             

    if (trial == "noninfer" and distr=="binomial"):

        factor1 = norm.ppf(1 - alpha) 

        if math.isnan(n):

            factor2 = norm.ppf(1 - beta) 
            n = 2 * (pi_val*(1 - pi_val) / delta**(2) ) * (factor1 + factor2)**2
            print("Estimated sample size (n): " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))
    
        elif math.isnan(power):
            diff = delta * (pi_val*(1-pi_val)*(2/n))**(-1/2) - factor1
            power = norm.cdf(diff, loc=0, scale=1) * 100
            print("Estimated power: " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))   
        
    if (trial == "equiv" and distr=="binomial"):
    
        factor1 = norm.ppf(1 - alpha) 
 
        if math.isnan(n):

            factor2 = norm.ppf(1 - beta/2) 
            n = 2 * (pi_val*(1 - pi_val) / delta**(2) ) * (factor1 + factor2)**2
            print("Estimated sample size (n): " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))
        
        elif math.isnan(power):
            power = 2 * norm.cdf(  (delta*(1-kappa*(2/n))) **(-1/2) - factor1 , loc=0, scale=1) - 1
            print("Estimated power: " +func_trial_description(trial_var=trial) + " trial for " + func_distribution_description(distr_var=distr) + " data ")
            return(func_res_dsn(trial=trial, distr=distr,  delta=delta, sigma=sigma, alpha=alpha, pi_val=pi_val,  n=n,  power=power))               


**Example 1 (a):** Estimate **sample size** for a noninferiority trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and  power=0.8

In [147]:
func_calc(trial = "noninfer", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05,
          pi_val=math.nan,  
          n = math.nan,  
          power = 0.8)

Estimated sample size (n): Non-Inferiority trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,noninfer,1.4,1.1,0.05,NaN,0.8,7.633566


**Example 1 (b):** Estimate **power** for a noninferiority trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and n=8

In [148]:
func_calc(trial = "noninfer", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan, 
          n = 8,  
          power = math.nan)

Estimated power: Non-Inferiority trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,noninfer,1.4,1.1,0.05,NaN,81.609973,8


**Example 2 (a):** Estimate **sample size** for a Equivalence trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and  power=0.8

In [149]:
func_calc(trial = "equiv", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan, 
          n = math.nan,  
          power = 0.8)

Estimated sample size (n): Equivalence trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,equiv,1.4,1.1,0.05,NaN,0.8,10.57373


**Example 2 (b):** Estimate **power** for a Equivalence trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and n=11

In [150]:
func_calc(trial = "equiv", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan, 
          n = 11,  
          power = math.nan)

Estimated power: Equivalence trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,equiv,1.4,1.1,0.05,NaN,81.974048,11


**Example 3 (a):** Estimate **sample size** for a Superiority trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and  power=0.8

In [151]:
func_calc(trial = "super", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan, 
          n = math.nan,  
          power = 0.8)

Estimated sample size (n): Superiority trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,super,1.4,1.1,0.05,NaN,0.8,7.633566


**Example 3 (b):** Estimate **power** for a Superiority trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and n=8

In [152]:
func_calc(trial = "super", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan, 
          n = 8,  
          power = math.nan)

Estimated power: Superiority trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,super,1.4,1.1,0.05,NaN,81.609973,8


**Example 4 (a):** Estimate **sample size** for a Superiority to Placebo trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and  power=0.8

In [153]:
func_calc(trial = "supertoplacebo", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan, 
          n = math.nan,  
          power = 0.8)

here
Estimated sample size (n): Superiority to Placebo trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,supertoplacebo,1.4,1.1,0.05,NaN,0.8,7.633566


**Example 4 (b):** Estimate **power** for a Superiority trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and n=8

In [154]:
func_calc(trial = "supertoplacebo", 
          distr = "normal", 
          delta = 1.4, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=math.nan,  
          n = 8,  
          power = math.nan)

here
Estimated power: Superiority to Placebo trial for normally distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,supertoplacebo,1.4,1.1,0.05,NaN,81.609973,8


**Example 5 (a):** Estimate **sample size** for a Noninferiority trial when data is binomially distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and  power=0.8

In [155]:
func_calc(trial = "noninfer", 
          distr = "binomial", 
          delta = .5, 
          pi_val= 0.2, 
          alpha = 0.05,
          sigma=math.nan,   
          n = math.nan,  
          power = 0.8)

Estimated sample size (n): Non-Inferiority trial for binomially distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,noninfer,0.5,NaN,0.05,0.2,0.8,7.913673


**Example 5 (b):** Estimate **power** for a Superiority trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and n=8

In [156]:
func_calc(trial = "noninfer", 
          distr = "binomial", 
          delta = 0.5, 
          sigma = 1.1, 
          alpha = 0.05, 
          pi_val=0.2,  
          n = 8,  
          power = math.nan)

Estimated power: Non-Inferiority trial for binomially distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,noninfer,0.5,1.1,0.05,0.2,80.376494,8


**Example 6 (a):** Estimate **sample size** for a Equivalent trial when data is binomially distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and  power=0.8

In [157]:
func_calc(trial = "noninfer", 
          distr = "binomial", 
          delta = .5, 
          pi_val= 0.2, 
          alpha = 0.05,
          sigma=math.nan, 
          n = math.nan,  
          power = 0.8)

Estimated sample size (n): Non-Inferiority trial for binomially distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,noninfer,0.5,NaN,0.05,0.2,0.8,7.913673


**Example 6 (b):** Estimate **power** for a Equivalent trial when data is normally distributed for the following parameters: $\delta$ = 1.4, sigma = 1.1, alpha ($\alpha$)=0.05 and n=8

In [158]:
func_calc(trial = "noninfer", 
          distr = "binomial", 
          delta = 0.5, 
          sigma = math.nan, 
          alpha = 0.05, 
          pi_val=0.2,  
          n = 8,  
          power = math.nan)

Estimated power: Non-Inferiority trial for binomially distributed data 


,trial,delta,sigma,alpha,pi_val,power,n
1,noninfer,0.5,NaN,0.05,0.2,80.376494,8
